In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

### META:
- Criar um sistema com cadastro de usuários e nível de acesso de cada um;
- Treinar um modelo de ML que me informará se devo ou não dar um empréstimo à ele;
- BONÚS: Dizer até qual valor seria seguro emprestar

### TRADUÇÃO DAS VARIAVEIS
-Annual Income == Rendimento Anual



In [43]:
df = pd.read_csv("/home/iago/Documentos/desafio-01/credit_risk_dataset.csv")

In [44]:
# Lendo os 5 primeiros dados
df.head(5)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [45]:
# Pegando a descrição de cada dado (quantidade de dados, valor médio, moda...)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,32581.0,27.734600,6.348078,20.00,23.00,26.00,30.00,144.00
person_income,32581.0,66074.848470,61983.119168,4000.00,38500.00,55000.00,79200.00,6000000.00
person_emp_length,31686.0,4.789686,4.142630,0.00,2.00,4.00,7.00,123.00
loan_amnt,32581.0,9589.371106,6322.086646,500.00,5000.00,8000.00,12200.00,35000.00
loan_int_rate,29465.0,11.011695,3.240459,5.42,7.90,10.99,13.47,23.22
loan_status,32581.0,0.218164,0.413006,0.00,0.00,0.00,0.00,1.00
loan_percent_income,32581.0,0.170203,0.106782,0.00,0.09,0.15,0.23,0.83
cb_person_cred_hist_length,32581.0,5.804211,4.055001,2.00,3.00,4.00,8.00,30.00


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [47]:
new_person_home = LabelEncoder()
new_loan_intent = LabelEncoder()
new_loan_grade = LabelEncoder()
new_cb_person_default_on_file = LabelEncoder()
df['person_home_ownership'] = new_person_home.fit_transform(df['person_home_ownership'])
df['loan_intent'] = new_loan_intent.fit_transform(df['loan_intent'])
df['loan_grade'] = new_loan_grade.fit_transform(df['loan_grade'])
df['cb_person_default_on_file'] = new_cb_person_default_on_file.fit_transform(df['cb_person_default_on_file'])

In [69]:
df.iloc[:, 3:]

,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,123.0,4,3,35000,16.02,1,0.59,1,3
1,5.0,1,1,1000,11.14,0,0.10,0,2
2,1.0,3,2,5500,12.87,1,0.57,0,3
3,4.0,3,2,35000,15.23,1,0.53,0,2
4,8.0,3,2,35000,14.27,1,0.55,1,4
...,...,...,...,...,...,...,...,...,...
32576,1.0,4,2,5800,13.16,0,0.11,0,30
32577,4.0,4,0,17625,7.49,0,0.15,0,19
32578,3.0,2,1,35000,10.99,1,0.46,0,28
32579,5.0,4,1,15000,11.48,0,0.10,0,26


In [ ]:
df2 = df.drop(columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'])

In [ ]:
df2.corr()['person_income']

person_age                    0.173202
person_income                 1.000000
person_emp_length             0.134268
loan_amnt                     0.266820
loan_int_rate                 0.000792
loan_status                  -0.144449
loan_percent_income          -0.254471
cb_person_cred_hist_length    0.117987
Name: person_income, dtype: float64

In [ ]:
df.value_counts().T

person_age  person_income  person_home_ownership  person_emp_length  loan_intent        loan_grade  loan_amnt  loan_int_rate  loan_status  loan_percent_income  cb_person_default_on_file  cb_person_cred_hist_length
32          65000          MORTGAGE               6.0                PERSONAL           B           5600       11.58          0            0.09                 N                          5                             2
25          78000          MORTGAGE               0.0                HOMEIMPROVEMENT    B           25000      9.64           0            0.32                 N                          4                             2
28          80004          MORTGAGE               5.0                MEDICAL            A           12000      6.91           0            0.15                 N                          6                             2
25          43200          OWN                    9.0                VENTURE            B           13800      11.48          0  

In [ ]:
df.shape

(32581, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [ ]:
df.corr()['person_income']

ValueError: could not convert string to float: 'RENT'